In [5]:
import csv
import os

In [7]:
# Variables
dataset_file = "extracted_tables/tables_regex.csv"
table_dir = "extracted_tables/table_code/"
header_dir = "extracted_tables/table_header/"
output_dir = "extracted_tables/table_images/"

with open(dataset_file, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='"')
    for row in spamreader:
        table_id = row[0]
        paper_id = row[1]
    
        with open(table_dir + table_id + ".txt", "r") as file_table:
            table = file_table.read()
        
        with open(header_dir + paper_id + ".txt", "r") as file_header:
            header = file_header.read()
        
        tex_code = header + "\n" + table + "\n\end{document}"
        #print(tex_code)
        code_path = output_dir + "table.tex"
        with open(code_path, "w") as file_code:
            file_code.write(tex_code)
        os.system(f"pdflatex -interaction=nonstopmode {code_path}")
        #os.remove(code_path)
    
        break